In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def remove_str(x):
    token1 = x.split(' złDo negocjacji')
    token2 = x.split(' zł')
    token3 = x.split(' zł/m²')
    token4 = x.split(' m²')
    if len(token1) == 2:
        return (str(token1[0]))
    elif len(token2) == 2:
        return (str(token2[0]))
    elif len(token3) == 2:
        return (str(token3[0]))
    elif len(token4) == 2:
        return (str(token4[0]))
    try:
        return str(x)
    except:
        return False    

In [ ]:
def rooms_number(x):
    if x == 'Kawalerka':
        return 1
    elif x == '2 pokoje':
        return 2
    elif x == '3 pokoje':
        return 3
    else:
        return 4

In [ ]:
def is_float(x):
    try:
        return float(x)
    except:
        return False    

In [ ]:
# What do I want to see:
1. 

# Analyzing

In [ ]:
olx_table = pd.read_csv('olx_offers.csv')

In [ ]:
olx_table['Offer_Type'] = np.where(olx_table['Czynsz (dodatkowo)'].isnull(), 'for_sale', 'for_rent')

In [ ]:
olx_table.drop(columns={'Finanse'}, axis='columns', inplace=True)

In [ ]:
olx_table = olx_table.rename(columns={'Cena': 'Price', 'Cena za m²': 'Price_m2', 'Czynsz (dodatkowo)': 'Rental (zł)', 
'Liczba pokoi': 'Rooms', 'Link': 'Link', 'Miejsce': 'Localisation', 'Oferta od': 'Owner', 'Powierzchnia': 'Living_Area (m2)',
'Poziom': 'Floor', 'Rodzaj zabudowy': 'Buildings_Type', 'Rynek': 'Market', 'Umeblowane': 'Furnishings', 'Miasto': 'City'})  # (rynek wtórny to aftermarket) choosing which columns do I need

In [ ]:
olx_table.head(2)

In [ ]:
# As we can see in the Price column we have some prices to negotiation. Let's create a column with info can we negotiate or no.
# Additionaly remove this info from the Price column. Keep just the value
olx_table['To_Negotiate'] = np.where(olx_table.Price.str.contains('Do negocjacji'), 'Y', 'N')

In [ ]:
olx_table['Price'] = olx_table['Price'].apply(remove_str)

In [ ]:
olx_table['Price_m2'] = olx_table['Price_m2'].replace(np.nan, '0  zł/m²')

In [ ]:
olx_table.iloc[0,1].split(' zł/m²')

In [ ]:
olx_table['Price_m2'] = olx_table['Price_m2'].apply(remove_str)

In [ ]:
olx_table['Price'] = olx_table['Price'].str.replace(' ', '') 

In [ ]:
olx_table['Price'] = olx_table['Price'].apply(is_float)

In [ ]:
olx_table['Price_m2'] = olx_table['Price_m2'].str.replace(' ', '') 

In [ ]:
olx_table['Price_m2'] = olx_table['Price_m2'].apply(is_float)

In [ ]:
olx_table['Living_Area (m2)'] = olx_table['Living_Area (m2)'].apply(remove_str)

In [ ]:
olx_table['Rooms'] = olx_table['Rooms'].apply(rooms_number)

In [ ]:
olx_r = olx_table[olx_table.Offer_Type=='for_rent'].copy()
olx_s = olx_table[olx_table.Offer_Type=='for_sale'].copy()

In [ ]:
olx_r.drop(columns={'Price_m2', 'Market'}, axis='columns', inplace=True) # we don't need a price_m2 and Market columns in this table

In [ ]:
olx_r['Rental (zł)'] = olx_r['Rental (zł)'].apply(remove_str)

In [ ]:
olx_s.drop(columns={'Rental (zł)'}, axis='columns', inplace=True) # we don't need a rental column in this table

In [ ]:
olx_s = olx_s.drop(olx_s[olx_s.Price==False].index) #There are some offers without prices e.g. exchange a flat. I removed them all

In [ ]:
len(olx_r)

In [ ]:
len(olx_s)

In [ ]:
olx_r.Floor.sort_values().unique()

In [ ]:
(olx_r.Floor.isnull().sum()) / len(olx_r) # just ~3% of the offers have no info about floor no. 

In [ ]:
olx_r.dropna(inplace=True)

In [ ]:
olx_s.head()

In [ ]:
olx_s.isnull().sum()

In [ ]:
olx_s.dropna(inplace=True)

In [ ]:
olx_s.isnull().sum()

# Charts

In [ ]:
def plot_bar_x(label1, label2):
    index = np.arange(len(label1))
    plt.bar(index, label2)
    plt.xlabel('', fontsize=10)
    plt.ylabel('Number of offers', fontsize=10)
    plt.xticks(index, label1, fontsize=10, rotation=30)
    plt.title('Amount of offers')
    plt.show()

In [ ]:
a = olx_r.groupby('Owner').agg({'Price': 'count'}).reset_index()
a

In [ ]:
# Create an empty list 
owners_label =[]
amount_label =[]
  
# Iterate over each row 
for index, rows in a.iterrows(): 
    owners_label.append(rows.Owner)
    amount_label.append(rows.Price)
  
print(owners_label,amount_label) 

In [ ]:
plot_bar_x(owners_label, amount_label)

What type of buildings do we have

In [ ]:
building_types = olx_r[olx_r.City=='Wodzislaw-Slaski'].groupby('Buildings_Type')['Price'].count().reset_index()

building_types_label = []
building_types_amount = []

for index, rows in building_types.iterrows():
    building_types_label.append(rows.Buildings_Type)
    building_types_amount.append(rows.Price)
print(building_types_label,building_types_amount)

In [ ]:
plot_bar_x(building_types_label, building_types_amount)

# Charts... in progress

In [ ]:
wodzislaw = olx_table[olx_table.City=='Wodzislaw-Slaski']

In [ ]:
rybnik = olx_table[olx_table.City=='Rybnik']

In [ ]:
plt.style.use('fivethirtyeight')

x = len(rybnik[rybnik.Offer_Type=='for_sale'])
y = len(rybnik[rybnik.Offer_Type=='for_rent'])
values_a = [x,y]
labels_a = ['For Sale', 'For Rent']
colors_a = ['#174F5F', '#3CAEA3']
explode_a = [0, 0.05]

i = len(wodzislaw[wodzislaw.Offer_Type=='for_sale'])
j = len(wodzislaw[wodzislaw.Offer_Type=='for_rent'])
values_b = [i,j]
labels_b = ['For Sale', 'For Rent']
colors_b = ['#174F5F', '#3CAEA3']
explode_b = [0, 0.05]

# create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2)

# plot each pie chart in a separate subplot
ax1.pie(values_a, labels=labels_a, colors=colors_a, explode=explode_a, startangle=0, autopct='%1.1f%%', wedgeprops={'edgecolor': 'black'})
ax2.pie(values_b, labels=labels_b, colors=colors_b, explode=explode_b, startangle=0, autopct='%1.1f%%', wedgeprops={'edgecolor': 'black'})

ax1.set_title('Rybnik')
ax2.set_title('Wodzislaw')
fig.suptitle('Offert Type Ratio')

#fig.legend(loc='upper center')
plt.tight_layout()
plt.show()